In this iPython notebook, we will featurize MOR ligand binding simulation by pairwise distances between the ligand and different receptor residues. We will then perform tICA and prospectively build an MSM. 

In [1]:
from PDB_Order_Fixer import PDB_Order_Fixer
import mdtraj as md
import os
import numpy as np
import h5py

import datetime
import glob
import copy
from functools import partial 
import operator
import time

import random 
import subprocess
from subprocess import Popen
import sys
from io_functions import *
from custom_clusterer import *
from custom_tica import *
from custom_featurizer import *
from pdb_editing import *
from analysis import *
from io_functions import *
#from topology_fixing import *
from subsampling import *
from conversions import *
from custom_msm import *
from grids import *

In [2]:
from detect_intermediates import *
from interpret_tICs import *

WHERE ARE WE?


In [3]:
from tutorial_feature_types import *
from get_variable_names import *
from tutorial_tica_config import *

tm6_tm3_residues
[A279, A165]
[65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 273, 274, 275, 276, 277, 278,

In [4]:
features_dir = "%s/ligand_protein_contacts" % (base)
print(features_dir)
if not os.path.exists(features_dir): os.makedirs(features_dir)
feature_residues_pkl = "%s/feature_residues.pkl" % features_dir
print(feature_residues_pkl)

/home/amir/Post_Process/GPCR/MOR/LIG_path/BU_path/ligand_protein_contacts
/home/amir/Post_Process/GPCR/MOR/LIG_path/BU_path/ligand_protein_contacts/feature_residues.pkl


In [5]:
ligand_residue = Residue(resSeq = 900, chain_id="J", res_name="LIG")

In [6]:
list(range(0,10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [60]:
excluded_set = set(list(range(270, 292)) + list(range(239, 263)) + list(range(153, 170)) + list(range(170, 197)) + list(range(102, 116)) + list(range(80, 102)) + list(range(330, 350)))
included_residues = [res for res in contact_residues if res.resSeq not in excluded_set]
user_specified_contact_residue_pairs = [(ligand_residue, contact_residue) for contact_residue in included_residues]
print(user_specified_contact_residue_pairs)

[(J900, I294), (J900, I324), (J900, I206), (J900, I140), (J900, I315), (J900, I66), (J900, I121), (J900, I212), (J900, I322), (J900, I235), (J900, I204), (J900, I123), (J900, I225), (J900, I229), (J900, I139), (J900, I296), (J900, I150), (J900, I128), (J900, I148), (J900, I197), (J900, I318), (J900, I129), (J900, I213), (J900, I152), (J900, I232), (J900, I149), (J900, I145), (J900, I298), (J900, I141), (J900, I238), (J900, I233), (J900, I223), (J900, I304), (J900, I301), (J900, I295), (J900, I221), (J900, I126), (J900, I203), (J900, I118), (J900, I78), (J900, I65), (J900, I151), (J900, I72), (J900, I76), (J900, I317), (J900, I70), (J900, I138), (J900, I220), (J900, I306), (J900, I316), (J900, I127), (J900, I293), (J900, I302), (J900, I227), (J900, I73), (J900, I137), (J900, I211), (J900, I218), (J900, I311), (J900, I200), (J900, I79), (J900, I132), (J900, I222), (J900, I75), (J900, I116), (J900, I313), (J900, I320), (J900, I305), (J900, I125), (J900, I130), (J900, I131), (J900, I142), 

In [61]:
os.system("rm %s" % feature_residues_pkl)
os.system("rm -rf %s" % features_dir)
if not os.path.exists(features_dir): os.makedirs(features_dir)
featurize_contacts_custom(traj_dir, features_dir = features_dir, traj_ext = traj_ext, contact_residue_pairs_file = feature_residues_pkl, structures=[], contact_residues=[],
                          residues_map = None, contact_cutoff = cutoff, parallel = featurize_parallel, exacycle = exacycle, traj_top_structure = None, iterative=False,
                          user_specified_contact_residue_pairs=user_specified_contact_residue_pairs)

There are 0 pairs of residues to be used in contact featurization.
Saving contact feature residue pairs to disk.
About to featurize trajectories based on the chosen featurization scheme.
looking at 132 pairs for trajectory
looking at 132 pairs for trajectory
looking at 132 pairs for trajectory
looking at 132 pairs for trajectory
looking at 132 pairs for trajectory
looking at 132 pairs for trajectory
looking at 132 pairs for trajectory
looking at 132 pairs for trajectory
new features /home/amir/Post_Process/GPCR/MOR/LIG_path/BU_path/h5_trajectories/BU-EQRep_3-0.h5 has shape: new features /home/amir/Post_Process/GPCR/MOR/LIG_path/BU_path/h5_trajectories/BU-EQRep_3-2.h5 has shape: new features /home/amir/Post_Process/GPCR/MOR/LIG_path/BU_path/h5_trajectories/BU-EQRep_2-0.h5 has shape: new features /home/amir/Post_Process/GPCR/MOR/LIG_path/BU_path/h5_trajectories/BU-EQRep_2-2.h5 has shape: new features /home/amir/Post_Process/GPCR/MOR/LIG_path/BU_path/h5_trajectories/BU-EQRep_1-0.h5 has sh

KeyboardInterrupt: 

In [20]:
import pickle
with open(feature_residues_pkl, "rb") as f:
    feature_residues = pickle.load(f)
print(feature_residues[0])
print(feature_residues[0][0].__dict__)
print(feature_residues[0][1].__dict__)
#tica_dir = "%s/tica_ligand_protein_contacts_2_sparse_0pt001" % base
tica_dir = "%s/tica_ligand_protein_contacts_2" % base


(J900, I304)
{'chain_name': None, 'ballosteros_weinstein': None, 'resSeq': 900, 'res_name': 'LIG', 'chain_id': 'J'}
{'chain_name': None, 'ballosteros_weinstein': None, 'resSeq': 304, 'res_name': 'ALA304', 'chain_id': 'I'}


In [9]:
os.system("rm -rf %s" % tica_dir)
lag_time = 5
n_components = 5
sparse = True
wolf = True
rho = 0.0001
shrinkage = None
traj_ext = ".h5"
fit_and_transform(features_directory = features_dir, model_dir = tica_dir, stride=5, lag_time = lag_time, n_components = n_components, sparse = sparse, wolf = wolf, rho = rho, shrinkage = shrinkage, parallel=True, traj_ext = traj_ext)

loading feature files


/home/amir/source/msmbuilder/msmbuilder/decomposition/sparsetica.py:149: ExperimentalWarning: 
""""""""""""""""""""""""""""""""""""""""""""""""""
""""""""""""""""""""""""""""""""""""""""""""""""""
  self._do_solve()
/home/amir/source/msmbuilder/msmbuilder/decomposition/speigh.py:392: UserWarning: optimal_inaccurate
  warnings.warn(problem.status)


(180, 128)
[ 1.09186065  0.50612396  1.24744558  1.43701756  0.50880903  1.40332401
  1.24323976  1.1783185   0.65211928  0.6267736 ]
(38,)
fitting data to tICA model
Sparse time-structure based Independent Components Analysis (tICA)
------------------------------------------------------------------
n_components        : 5
gamma               : 0.05
lag_time            : 5
weighted_transform  : True
rho                 : 0.0001
n_features          : 128

Top 5 timescales :
[ 0.  0.  0.  0.  0.]

Top 5 eigenvalues :
[ 0.  0.  0.  0.  0.]

Number of active degrees of freedom:
[0/128, 0/128, 0/128, 0/128, 0/128]
loading /home/amir/Post_Process/GPCR/MOR/LIG_path/BU_path/ligand_protein_contacts/BU-EQRep_0-0.dataset
loading /home/amir/Post_Process/GPCR/MOR/LIG_path/BU_path/ligand_protein_contacts/BU-EQRep_2-2.dataset
loading /home/amir/Post_Process/GPCR/MOR/LIG_path/BU_path/ligand_protein_contacts/BU-EQRep_2-0.dataset
loading /home/amir/Post_Process/GPCR/MOR/LIG_path/BU_path/ligand_protein_c

/home/amir/source/msmbuilder/msmbuilder/decomposition/speigh.py:392: UserWarning: unbounded_inaccurate
  warnings.warn(problem.status)


In [11]:
projected_features_dir = "%s/phi_psi_chi2_allprot_projected.h5" % tica_dir
#plot_columns(tica_dir, projected_features_dir, titles = ["tIC%d" %j for j in range(1,6)], tICA = True, scale = 1.0, refcoords_file = None)

In [17]:
%load_ext autoreload
%autoreload 2

In [12]:
tic_components_dir = "%s/tic_components" % tica_dir
if not os.path.exists(tic_components_dir): os.makedirs(tic_components_dir)
projection_operator_dir = "%s/phi_psi_chi2_allprot_tica_coords.h5" % tica_dir
#interpret_tIC_components(projection_operator_dir, tic_components_dir, feature_residues_pkl, n_tica_components=5, percentile=95)


In [12]:
n_clusters = 20
clusterer_dir = "%s/clusterer_20clusters.h5" % tica_dir
cluster_minikmeans(tica_dir, projected_features_dir, traj_dir, n_clusters, clusterer_dir, tICs=list(range(0,5)))

Clustering by KMeans
loading "/home/amir/Post_Process/GPCR/MOR/LIG_path/BU_path/tica_ligand_protein_contacts/phi_psi_chi2_allprot_projected.h5"...
Saving "/home/amir/Post_Process/GPCR/MOR/LIG_path/BU_path/tica_ligand_protein_contacts/clusterer_20clusters.h5"... (<class 'msmbuilder.cluster.MiniBatchKMeans'>)


In [13]:
n_samples=10
save_dir = "%s/clusters20_samples10" % tica_dir
sampling_method = "random"
clusters_map_file = "%s/clusters20_map.h5" % tica_dir
sample_clusters(clusterer_dir, projected_features_dir, traj_dir, traj_ext, save_dir, n_samples, method = sampling_method, clusters_map_file = clusters_map_file)

loading "/home/amir/Post_Process/GPCR/MOR/LIG_path/BU_path/tica_ligand_protein_contacts/clusterer_20clusters.h5"...
262
634
324
508
230
218
53
495
127
51
199
127
157
225
196
46
170
213
73
99
loading "/home/amir/Post_Process/GPCR/MOR/LIG_path/BU_path/tica_ligand_protein_contacts/phi_psi_chi2_allprot_projected.h5"...
[-0.01124774  0.58449434 -1.08396697  1.11620643 -0.65247929]
[-0.67914499  0.67845379  0.12448838 -0.51431626  1.13534581]
[ 0.61522346 -1.69544941 -0.25643443 -0.33323974  0.26822238]
[-0.54891778  0.45493771  0.0723318   0.12042543  0.07938815]
[-0.40135866 -0.42609819  1.43762083 -1.24069908 -1.50903686]
[-0.05751177 -0.35714849 -0.27142278  1.34217091 -0.0967051 ]
[ 2.32771449  2.74901408  1.77317397  2.60938902  0.24579139]
[-0.65404158  0.98874826 -0.6921161  -0.97706034 -0.64371765]
[ 0.53989198 -1.17048726 -0.77504116  0.220729   -0.6812312 ]
[ 5.03829811  2.25991895  0.87660835 -2.2717616   1.20413478]
[-0.16659334 -0.45086256  0.68093974  0.51715303  0.14358958]
[

In [ ]:
plot_all_tics_and_clusters(tica_dir, projected_features_dir, clusterer_dir, lag_time, label = "dot", active_cluster_ids = range(0,20), intermediate_cluster_ids = [], inactive_cluster_ids = [])

In [13]:
feature_files = get_trajectory_files(features_dir, ".dataset")
features = [load_file(feature_file) for feature_file in feature_files]

loading /home/amir/Post_Process/GPCR/MOR/LIG_path/BU_path/ligand_protein_contacts/BU-EQRep_0-0.dataset
loading /home/amir/Post_Process/GPCR/MOR/LIG_path/BU_path/ligand_protein_contacts/BU-EQRep_0-1.dataset
loading /home/amir/Post_Process/GPCR/MOR/LIG_path/BU_path/ligand_protein_contacts/BU-EQRep_0-2.dataset
loading /home/amir/Post_Process/GPCR/MOR/LIG_path/BU_path/ligand_protein_contacts/BU-EQRep_0-3.dataset
loading /home/amir/Post_Process/GPCR/MOR/LIG_path/BU_path/ligand_protein_contacts/BU-EQRep_1-0.dataset
loading /home/amir/Post_Process/GPCR/MOR/LIG_path/BU_path/ligand_protein_contacts/BU-EQRep_1-1.dataset
loading /home/amir/Post_Process/GPCR/MOR/LIG_path/BU_path/ligand_protein_contacts/BU-EQRep_1-2.dataset
loading /home/amir/Post_Process/GPCR/MOR/LIG_path/BU_path/ligand_protein_contacts/BU-EQRep_1-3.dataset
loading /home/amir/Post_Process/GPCR/MOR/LIG_path/BU_path/ligand_protein_contacts/BU-EQRep_2-0.dataset
loading /home/amir/Post_Process/GPCR/MOR/LIG_path/BU_path/ligand_protein_

In [14]:
features = np.concatenate(features)

In [18]:
tics = np.concatenate(load_file(projected_features_dir))
print(np.shape(tics))

loading /home/amir/Post_Process/GPCR/MOR/LIG_path/BU_path/tica_ligand_protein_contacts_2/phi_psi_chi2_allprot_projected.h5
loading "/home/amir/Post_Process/GPCR/MOR/LIG_path/BU_path/tica_ligand_protein_contacts_2/phi_psi_chi2_allprot_projected.h5"...
(6060, 5)


In [21]:
with open(feature_residues_pkl, "rb") as f:
    feature_residues = pickle.load(f)

In [45]:
feature_to_column = {}
for j, feature in enumerate(feature_residues):
    feature_to_column[feature[1].resSeq] = j

In [25]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [59]:
#plt.hexbin(tics[:,0], features[:,feature_to_column[324]], bins='log')
plt.hexbin(features[:,feature_to_column[147]], features[:,feature_to_column[326]], bins='log')

#plt.axis([-1, 2, 0, 1])

KeyError: 326

In [48]:
subset = crystal_structure.atom_slice(range(0,400))
subset.xyz
print(subset.xyz)
distances = md.compute_contacts(subset)
print(distances)


[[[-3.67750001 -2.00090003 -3.06229997]
  [-3.55780005 -1.92980003 -3.01830006]
  [-3.56550002 -1.78180003 -3.05369997]
  ..., 
  [-1.46140003 -0.32269999 -1.54359996]
  [-1.40170002 -0.40900001 -1.77069998]
  [-1.92920005 -0.43360001 -1.81420004]]]
(array([[ 0.29087129,  0.30480972,  0.50911838, ...,  0.31547278,
         0.29779878,  0.30303043]], dtype=float32), array([[ 0,  3],
       [ 0,  4],
       [ 0,  5],
       ..., 
       [47, 50],
       [47, 51],
       [48, 51]]))


In [20]:
distances0 = distances[0]

In [23]:
distances0 = np.nan_to_num(distances0)

In [24]:
distances0

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [25]:
np.where(distances0 > 0.0)

(array([], dtype=int64), array([], dtype=int64))